# Basic Tutorial III: Functional Workflow
## Introduction

**[AutoRA](https://pypi.org/project/autora/)** (**Au**tomated **R**esearch **A**ssistant) is an open-source framework designed to automate various stages of empirical research, including model discovery, experimental design, and data collection.

This notebook is the third of four notebooks within the basic tutorials of ``autora``. We suggest that you go through these notebooks in order as each builds upon the last. However, each notebook is self-contained and so there is no need to *run* the content of the last notebook for your current notebook.

These notebooks provide a comprehensive introduction to the capabilities of ``autora``. **It demonstrates the fundamental components of ``autora``, and how they can be combined to facilitate automated (closed-loop) empirical research through synthetic experiments.**

**How to use this notebook** *You can progress through the notebook section by section or directly navigate to specific sections. If you choose the latter, it is recommended to execute all cells in the notebook initially, allowing you to easily rerun the cells in each section later without issues.*

### Tutorial Setup

We will here import some standard python packages, set seeds for replicability, and define a plotting function.

In [ ]:
#### Installation ####
!pip install -q "autora[theorist-bms]"
!pip install -q "autora[experiment-runner-synthetic-abstract-equation]"

#### Import modules ####
from typing import Optional
import numpy as np
import pandas as pd
import sympy as sp
import torch
import matplotlib.pyplot as plt

from autora.state import StandardState

#### Set seeds ####
np.random.seed(42)
torch.manual_seed(42)

#### Define plot function ####
def plot_from_state(s: StandardState, expr: str):    
    
    """
    Plots the data, the ground truth model, and the current predicted model
    """
    
    #Determine labels and variables
    model_label = f"Model: {s.models[-1]}" if hasattr(s.models[-1],'repr') else "Model"
    experiment_data = s.experiment_data.sort_values(by=["x"])
    ground_x = np.linspace(s.variables.independent_variables[0].value_range[0],s.variables.independent_variables[0].value_range[1],100)
    
    #Determine predicted ground truth
    equation = sp.simplify(expr)
    ground_predicted_y = [equation.evalf(subs={'x':x}) for x in ground_x]
    model_predicted_y = s.models[-1].predict(ground_x.reshape(-1, 1))

    #Plot the data and models
    f = plt.figure(figsize=(4,3))
    plt.plot(experiment_data["x"], experiment_data["y"], 'o', label = None)
    plt.plot(ground_x, model_predicted_y, alpha=.8, label=model_label)
    plt.plot(ground_x, ground_predicted_y, alpha=.8,  label=f'Ground Truth: {expr}')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.legend()
    plt.show()

## States

Using the functions and objects in `autora.state`, we can build flexible pipelines and cycles which operate on state
objects. State objects represent data from an experiment, like the conditions, observed experiment data and models. The State also includes rules on how to update those data if new data are provided using the "Delta mechanism". This state can be acted upon by experimentalists, experiment runners, and theorists. 

In tutorial I, we had experimentalists define new conditions, experiment runners collect new observations, and theorists model the data. To do this, we used the output of one as the input of the other, such as: 

`conditions = experimentalist(...)` $\rightarrow$ <br>
`observations = experiment_runner(conditions,...)` $\rightarrow$ <br>
`model = theorist(conditions, observations)` <br>

This chaining is embedded within the `State` functionality. To act on a state, we must wrap each of our experimentalist(s), experiment_runner(s), and theorist(s) so that they:
- operate on the `State`, and
- return a modified object of the **same type** `State`.

### Defining The State

We use the `StandardState` object. Let's begin by populating the state with *variable information* (`variables`), *seed condition data* (`conditions`), and a *dataframe* (`pd.DataFrame(columns=["x","y"])`) that will hold our conditions (`x`) and observations (`y`).

*Note: Some `autora` components have a `random_state` parameter that sets the seed for random number generators. Using this parameter ensures reproducibility of your code, but is optional.*

In [ ]:
from autora.variable import Variable, ValueType, VariableCollection
from autora.experimentalist.random import random_pool
from autora.state import StandardState

#### Define variable data ####
iv = Variable(name="x", value_range=(0, 2 * np.pi), allowed_values=np.linspace(0, 2 * np.pi, 30))
dv = Variable(name="y", type=ValueType.REAL)
variables = VariableCollection(independent_variables=[iv],dependent_variables=[dv])

#### Define seed condition data ####
conditions = random_pool(variables, num_samples=10, random_state=0)

#### Initialize State ####
s = StandardState(
    variables = variables,
    conditions = conditions,
    experiment_data = pd.DataFrame(columns=["x","y"])
)

### Viewing the State

Now, let's view the contents of the state we just initialized.

In [ ]:
print(s)

We can view all of the content we provided the state more directly if we choose.

In [ ]:
print("\033[1mThe variables we provided:\033[0m")
print(s.variables)

print("\n\033[1mThe conditions we provided:\033[0m")
print(s.conditions)

print("\n\033[1mThe dataframe we provided:\033[0m")
print(s.experiment_data)

### AutoRA Components and the State

Now that we have initialized the state, we need to start preparing components of `autora` to work with the state - namely, experimentalists, experiment runners, and theorists. 

These components are defined in the same way as past tutorials. All we need to do so that these can function within the state is to wrap them in specialized state functions. Note that as the theorists are using the `scikit-learn` interface, this will need to be wrapped differently than the experiment runners and experimentalists. The wrappers are:
- `on_state()` for experiment runners and experimentalists
- `estimator_on_state()` for theorists (specifically, scikit-learn estimators)

The first argument for each wrapper should be your corresponding function (i.e., the experiment runner, the experimentalist, and the theorist). The `on_state` wrapper takes a second argument, `output`, to determine where in the state the component is acting on. For the experimentalist this will be `output=["conditions"]`, and for the experiment runner this will be `output=["experiment_data"]`.

Once the components are wrapped, their functionality changes to act on the state, meaning that they now expect a state as the first input and will return a modified version of that state.

## Wrapping Components to Work with State

### Experimentalist Defined and Wrapped with State

We will use autora's `random_pool` pooler for our experimentalist. We import this and then wrap it so that it functions with the state.

In [ ]:
from autora.experimentalist.random import random_pool
from autora.state import on_state

experimentalist = on_state(random_pool, output=["conditions"])

### Experiment Runner Defined and Wrapped with State
We define a sine experiment runner and then wrap it so that it functions with the state.

To create our experiment runner, we will use an `AutoRA` function called `equation_experiment()`. This function takes in an equation wrapped as a `sympy` object using `sp.simplify()` and then allows us to solve for any input (`x`) given. Further, we constrain the values that this function can output by passing it the `variable` information.

In [ ]:
import sympy as sp
from autora.experiment_runner.synthetic.abstract.equation import equation_experiment
from autora.state import on_state

#### Define variable data ####
iv = Variable(name="x", value_range=(0, 2 * np.pi), allowed_values=np.linspace(0, 2 * np.pi, 30))
dv = Variable(name="y", type=ValueType.REAL)
variables = VariableCollection(independent_variables=[iv],dependent_variables=[dv])

#### Equation Experiment Method ####
sin_experiment = equation_experiment(sp.simplify('sin(x)'), variables.independent_variables, variables.dependent_variables[0])
sin_runner = sin_experiment.run

experiment_runner = on_state(sin_runner, output=["experiment_data"])

### Theorist Defined and Wrapped with State

We will use autora's `BMSRegressor` theorist. We import this and then wrap it so that if functions with the state.

In [ ]:
from autora.theorist.bms import BMSRegressor
from autora.state import estimator_on_state

theorist = estimator_on_state(BMSRegressor(epochs=100))

## Running Each Component with the State

### Run the Experimentalist

Let's run the experimentalist with the state and see how the state changes.

In [ ]:
print('\033[1mPrevious Conditions:\033[0m')
print(s.conditions)

s = experimentalist(s, num_samples=10, random_state=42)

print('\n\033[1mUpdated Conditions:\033[0m')
print(s.conditions)

### Run the Experiment Runner

Let's run the experiment runner and see how the state changes.

In [ ]:
print("\033[1mPrevious Data:\033[0m")
print(s.experiment_data)

s = experiment_runner(s, added_noise=1.0, random_state=42)

print("\n\033[1mUpdated Data:\033[0m")
print(s.experiment_data)

### Run the Theorist

Let's run the theorist and see how the state changes. Note that theorists should return a single model directly or multiple models within a list to work with the state.

In [ ]:
s = theorist(s)

print("\n\033[1mUpdated Model:\033[0m")
print(s.models[-1])

plot_from_state(s,'sin(x)')

### Component Chaining

As such, we have our `autora` components wrapped to work with the state. Remember, this means that they take the state as an input and returns the updated state as an output. As the components all act on the state, they can easily be chained.

In [ ]:
s = experimentalist(s, num_samples=10, random_state=42)
s = experiment_runner(s, added_noise=1.0, random_state=42)
s = theorist(s)

print(s)
plot_from_state(s,'sin(x)')

## The Cycle

Moreover, we can use these chained components within a loop to run multiple cycles.

### Cycle using Number of Cycles

In [ ]:
#### First, let's reinitialize the state object to get a clean state ####
iv = Variable(name="x", value_range=(0, 2 * np.pi), allowed_values=np.linspace(0, 2 * np.pi, 30))
dv = Variable(name="y", type=ValueType.REAL)
variables = VariableCollection(independent_variables=[iv],dependent_variables=[dv])

conditions = random_pool(variables, num_samples=10, random_state=42)

s = StandardState(
    variables = variables,
    conditions = conditions,
    experiment_data = pd.DataFrame(columns=["x","y"])
)

### Then we cycle through the pipeline we built three more times ###
num_cycles = 3 # number of empirical research cycles
for cycle in range(num_cycles):
    #Run pipeline
    s = experimentalist(s, num_samples=10, random_state=42+cycle)
    s = experiment_runner(s, added_noise=1.0, random_state=42+cycle)
    s = theorist(s)
    
    #Report metrics
    print(f"\n\033[1mRunning Cycle {cycle+1}:\033[0m")
    print(f"\033[1mCycle {cycle+1} model: {s.models[-1]}\033[0m")
    plot_from_state(s,'sin(x)')

### Cycle using Stopping Criteria

Alternatively, we can run the chain until we reach a stopping criterion. For example, here we will loop until we get 40 datapoints.

In [ ]:
#### First, let's reinitialize the state object to get a clean state ####
iv = Variable(name="x", value_range=(0, 2 * np.pi), allowed_values=np.linspace(0, 2 * np.pi, 30))
dv = Variable(name="y", type=ValueType.REAL)
variables = VariableCollection(independent_variables=[iv],dependent_variables=[dv])

conditions = random_pool(variables, num_samples=10, random_state=42)

s = StandardState(
    variables = variables,
    conditions = conditions,
    experiment_data = pd.DataFrame(columns=["x","y"])
)

### Then we cycle through the pipeline we built until we reach our stopping criterion ###
cycle = 0
while len(s.experiment_data) < 40: #Run until we have at least 40 datapoints
    #Run pipeline
    s = experimentalist(s, num_samples=10, random_state=42+cycle)
    s = experiment_runner(s, added_noise=1.0, random_state=42+cycle)
    s = theorist(s)
    
    #Report metrics
    print(f"\n\033[1mRunning Cycle {cycle+1}, number of datapoints: {len(s.experiment_data)}\033[0m")
    print(f"\033[1mCycle {cycle+1} model: {s.models[-1]}\033[0m")
    plot_from_state(s,'sin(x)')
    
    #Increase count
    cycle += 1

print(f"\n\033[1mNumber of datapoints: {len(s.experiment_data)}\033[0m")
print(f"\033[1mDetermined Model: {s.models[-1]}\033[0m")

### The Conditional Cycle 

Because `autora` components (theorist, experiment runner, experimentalist) act on the state, building a pipeline can have a lot of flexibility. Above, we demonstrated using a single set of components in different loops, but the components can also change respective to your criteria. In other words, you can use `if-else` statements to control which component is acting on the state.

For example, we can choose a different experimentalist depending on the number of datapoints we have collected.

In [ ]:
#### We will first define a new experimentalist
def uniform_sample(variables: VariableCollection, conditions: pd.DataFrame, num_samples: int = 1, random_state: Optional[int] = None):

    """
    An experimentalist that selects the least represented datapoints
    """

    #Set random state
    rng = np.random.default_rng(random_state)
    
    #Retrieve the possible values
    allowed_values = variables.independent_variables[0].allowed_values
    
    #Determine the representation of each value
    conditions_count = np.array([conditions["x"].isin([value]).sum(axis=0) for value in allowed_values])
    
    #Sort to determine the least represented values
    conditions_sort = conditions_count.argsort()
    
    conditions_count = conditions_count[conditions_sort]
    values_count = allowed_values[conditions_sort]
    
    #Sample from values with the smallest frequency
    x = values_count[conditions_count<=conditions_count[num_samples-1]]
    x = rng.choice(x,num_samples)
    
    return pd.DataFrame({"x": x})

In [ ]:
from autora.experimentalist.random import random_pool

#### First, let's reinitialize the state object to get a clean state ####
iv = Variable(name="x", value_range=(0, 2 * np.pi), allowed_values=np.linspace(0, 2 * np.pi, 30))
dv = Variable(name="y", type=ValueType.REAL)
variables = VariableCollection(independent_variables=[iv],dependent_variables=[dv])

conditions = random_pool(variables, num_samples=10, random_state=42)

s = StandardState(
    variables = variables,
    conditions = conditions,
    experiment_data = pd.DataFrame(columns=["x","y"])
)

#### Initiate both experimentalists ####
uniform_experimentalist = on_state(uniform_sample, output=["conditions"])
random_experimentalist = on_state(random_pool, output=['conditions'])

### Then we cycle through the pipeline we built until we reach our stopping criteria ###
cycle = 0
while len(s.experiment_data) < 40:
    
    #Run pipeline
    if len(s.experiment_data) < 20: #Conditional experimentalist: random for first half of cyles
        print('\n#==================================================#')
        print('\033[1mUsing random pooler experimentalist...\033[0m')
        s = random_experimentalist(s, num_samples=10, random_state=42+cycle)
    else: #Conditional experimentalist: uniform for last half of cycles
        print('\n#==================================================#')
        print('\033[1mUsing uniform sampler experimentalist...\033[0m')
        s = uniform_experimentalist(s, num_samples=10, random_state=42+cycle)
        
    s = experiment_runner(s, added_noise=1.0, random_state=42+cycle)
    s = theorist(s)
    
    #Report metrics
    print(f"\n\033[1mRunning Cycle {cycle+1}:\033[0m")
    print(f"\033[1mCycle {cycle+1} model: {s.models[-1]}\033[0m")
    plot_from_state(s,'sin(x)')
    
    #Increase count
    cycle += 1

In addition, we can dynamically change parameters across cycles.

In [ ]:
from autora.experimentalist.random import random_pool

#### First, let's reinitialize the state object to get a clean state ####
iv = Variable(name="x", value_range=(0, 2 * np.pi), allowed_values=np.linspace(0, 2 * np.pi, 30))
dv = Variable(name="y", type=ValueType.REAL)
variables = VariableCollection(independent_variables=[iv],dependent_variables=[dv])

conditions = random_pool(variables, num_samples=10, random_state=42)

s = StandardState(
    variables = variables,
    conditions = conditions,
    experiment_data = pd.DataFrame(columns=["x","y"])
)

#### Initiate both experimentalists ####
random_experimentalist = on_state(random_pool, output=['conditions'])

### Then we cycle through the pipeline we built until we reach our stopping criteria ###
for cycle, num_samples in enumerate([5, 10, 20, 50, 100]):
    
    #Report metrics
    print(f"\n\033[1mRunning Cycle {cycle+1} with {num_samples} new samples:\033[0m")

    #Run pipeline
    s = random_experimentalist(s, num_samples=num_samples, random_state=42+cycle)
    s = experiment_runner(s, added_noise=1.0, random_state=42+cycle)
    s = theorist(s)
    
    #Report metrics
    print(f"\033[1mCycle {cycle+1} model: {s.models[-1]}\033[0m")
    plot_from_state(s,'sin(x)')

## Next Notebook
This concludes the tutorial on ``autora`` functionality. However, ``autora`` is a flexible framework in which users can integrate their own theorists, experimentalists, and experiment_runners in an automated empirical research workflow. The next notebook, **AutoRA Basic Tutorial IV: Customization**, illustrates how to add your own custom theorists and experimentalists to use with ``autora``.